In [ ]:
import os
import h5py

import numpy as np
import xml.etree.ElementTree as et 
from collections import Counter, defaultdict

%matplotlib inline
import matplotlib.pyplot as plt

DATA_DIRECTORY = '/mnt/disks/sax-lax-40k-lvm/2020-01-29/'

In [ ]:
def random_copy_hd5_datasets(source_hd5, source_hd6, destination_hd5, group_path='/'):
    for k in source_hd5[group_path]:
        if isinstance(source_hd5[group_path][k], h5py.Dataset):
            dice = np.random.rand()
            if dice > 0.5 and group_path + k in source_hd6:
                if source_hd6[group_path][k].chunks is None:
                    destination_hd5.create_dataset(group_path + k, data=source_hd6[group_path][k])
                else:
                    destination_hd5.create_dataset(group_path + k, data=source_hd6[group_path][k], compression='gzip')
            else:
                if source_hd5[group_path][k].chunks is None:
                    destination_hd5.create_dataset(group_path + k, data=source_hd5[group_path][k])
                else:
                    destination_hd5.create_dataset(group_path + k, data=source_hd5[group_path][k], compression='gzip')                
        else:
            #logging.debug(f"copying group {group_path + k}")
            random_copy_hd5_datasets(source_hd5, source_hd6, destination_hd5, group_path=group_path + k + '/')

In [ ]:
files = os.listdir(DATA_DIRECTORY)
for i,file1 in enumerate(files):
    file2 = files[i+1]
    if file1[-4:] != '.hd5' or file2[-4:] != '.hd5':
        continue
    if i > 10:
        break
    with h5py.File(os.path.join(DATA_DIRECTORY, file1)) as hd5:
        with h5py.File(os.path.join(DATA_DIRECTORY, file2)) as hd6:
            with h5py.File(f'fake_{i}.hd5', 'w') as hdfake:
                random_copy_hd5_datasets(hd5, hd6, hdfake)